<a href="https://colab.research.google.com/github/Ilmaj17/Hotel_Risk_Classifier/blob/main/Hotel_Risk_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [146]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


###Determining whether it is a technical issue or a issue related to service quality

In [147]:
df = pd.read_csv("/content/drive/MyDrive/Data Sets/Capsule Hotel Risk.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          700 non-null    int64 
 1   content             700 non-null    object
 2   translated_content  700 non-null    object
 3   priority_score      700 non-null    int64 
 4   problem_domain      700 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 27.5+ KB


In [148]:
df.head(5)

,Unnamed: 0,content,translated_content,priority_score,problem_domain
0,1,"Sorry, including the expensive price + apk usi...","Sorry, including the expensive price + apk usi...",2,0
1,2,Setelah install terus pas buka aplikasi ngebla...,"After installed, the app stuck with white display",3,1
2,3,"gak bisa submit pada saat verifikasi data, pad...","Cannot submit the data verification, even my i...",3,1
3,4,Aplilasi restart mulu,The app keep restarting,3,1
4,5,"udah mahal , makanan ga enak, banyak lalat","Overpriced, bad food, too many flies",2,0


###Pipeline


*   Remove any unwanted columns
*   Converting categorical columns to numerical columns
*   Classification Model





In [149]:
from sklearn.base import BaseEstimator, TransformerMixin

In [150]:
#Creating an estimator by extending BaseEstimator class
class NameDropper(BaseEstimator, TransformerMixin):
  def fit(self, X, y=None):
    return self
  def transform(self, X):
    return X.drop(['content'], axis=1)

df2 = df
#dropper = NameDropper()
#df2 = dropper.fit_transform(df2)
#df.head()

In [151]:
df['translated_content'].nunique()

408

In [152]:
df['priority_score'].nunique()

3

###Applying Label encoding tp convert translated_content to numerical values

In [153]:
from sklearn.preprocessing import LabelEncoder
class FeatureEncoder(BaseEstimator, TransformerMixin):
  def fit(self, X, y=None):
    return self
  def transform(self, X):
    lb = LabelEncoder()
    X['translated_content'] = lb.fit_transform(X['translated_content'])
    return X
#enc = FeatureEncoder()
#enc.fit_transform(df)
#df.head()

In [154]:
#creating the pipeline
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
pipe = Pipeline([("dropper", NameDropper()), ("encoder" , FeatureEncoder()), ("dtc", DecisionTreeClassifier())])

In [155]:
#visualizing the pipeline
from sklearn import set_config
set_config(display="diagram")
pipe

Pipeline(steps=[('dropper', NameDropper()), ('encoder', FeatureEncoder()),
                ('dtc', DecisionTreeClassifier())])

In [156]:
#df2 = pipe.fit_transform(df2)
#df2 = pd.DataFrame(df2)
#print(type(df2))
#df2.head()
#df2

In [157]:
df.shape

(700, 5)

In [158]:
X = df.drop('problem_domain', axis = 1)
y = df['problem_domain']
from sklearn.model_selection import train_test_split
X_train,  X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 2)
y_train.head()

328    0
627    1
231    0
226    0
637    0
Name: problem_domain, dtype: int64

In [159]:
pipe.fit(X_train, y_train)
#X_train.head()
#X_train.head()
#df.head()

Pipeline(steps=[('dropper', NameDropper()), ('encoder', FeatureEncoder()),
                ('dtc', DecisionTreeClassifier())])

In [160]:
#X_train.shape

In [161]:
y_predict = pipe.predict(X_test)
y_predict.shape
#X_test.head()

(140,)

In [162]:
#from sklearn.tree import DecisionTreeClassifier
#dtc = DecisionTreeClassifier(random_state = 50)
#dtc.fit(X_train, y_train)

In [163]:
#y_predict = dtc.predict(X_test)
#y_predict

In [164]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
report = classification_report(y_test, y_predict)
print(report)

              precision    recall  f1-score   support

           0       0.89      0.80      0.84       114
           1       0.39      0.58      0.47        26

    accuracy                           0.76       140
   macro avg       0.64      0.69      0.66       140
weighted avg       0.80      0.76      0.77       140



In [165]:
accuracy = accuracy_score(y_test, y_predict)
print(accuracy)

0.7571428571428571


In [167]:
cm = confusion_matrix(y_test, y_predict)
cm

array([[91, 23],
       [11, 15]])